In [1]:
"""
Create PFT files for Ingrid

The rule is that PCT_GLACIER + PCT_URBAN + PCT_LAKE + PCT_WETLAND + PCT_NATVEG + PCT_CROP = 100%
If the sum total of these in the data is more than 100%, the excess is first subtracted from PCT_NATVEG.  If PCT_NATVEG becomes zero and there is still excess, then remaining excess is subtracted from PCT_CROP.  If there is still excess, those are subtracted in the following priority order:  PCT_URBAN, PCT_WETLAND,  PCT_LAKE, PCT_GLACIER.

PFTs that are of interest:
    1 not_vegetated
    2 needleleaf_evergreen_temperate_tree + needleleaf_evergreen_boreal_tree
    3 broadleaf_deciduous_temperate_tree + broadleaf_deciduous_boreal_tree
    4 broadleaf_evergreen_shrub + broadleaf_deciduous_temperate_shrub
            + broadleaf_deciduous_boreal_shrub
    5 c3_arctic_grass + c3_non-arctic_grass + c4_grass
    6 c3_crop
"""
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import cartopy.crs as ccrs
import rasterio
from rasterio.transform import from_origin

path_surfdata = os.path.join(
    os.environ['E3SM_ROOT'], 'inputdata', 'lnd', 'clm2', 'surfdata_map'
)

In [2]:
def netcdf_to_tif(file_in, year, file_out):
    hr = xr.open_dataset(file_in)

    pct_glaier = hr['PCT_GLACIER'].values
    pct_urban = hr['PCT_URBAN'].sum(axis = 0).values
    pct_lake = hr['PCT_LAKE'].values
    pct_wetland = hr['PCT_WETLAND'].values
    pct_natveg = hr['PCT_NATVEG'].values
    pct_crop = hr['PCT_CROP'].values

    tot_frac = pct_glaier + pct_urban + pct_lake + pct_wetland + pct_natveg + pct_crop
    pct_natveg = np.clip(np.where(tot_frac > 100, pct_natveg - (tot_frac-100), pct_natveg), 
                        a_min = 0., a_max = 100.)
    tot_frac = pct_glaier + pct_urban + pct_lake + pct_wetland + pct_natveg + pct_crop
    pct_crop = np.clip(np.where(tot_frac > 100, pct_crop - (tot_frac-100), pct_crop), 
                    a_min = 0., a_max = 100.)
    tot_frac = pct_glaier + pct_urban + pct_lake + pct_wetland + pct_natveg + pct_crop

    """
    # Diagnostic of the above procedure - works

    # PCT_NATVEG ==============================
    fig, ax = plt.subplots(1, 1, figsize = (15, 10), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines()
    ax.set_extent([-140, -50, 20, 55])
    cf = ax.contourf(
        hr['LONGXY'][0, :].values, hr['LATIXY'][:, 0].values,
        pct_natveg, levels = np.linspace(0, 100., 21),
                    norm = BoundaryNorm(np.linspace(0, 100., 21), ncolors = 256, 
                                        clip = True), cmap = 'Reds')
    plt.colorbar(cf, ax = ax, shrink = 0.7)
    ax.set_title('PCT_NATVEG')

    # PCT_CROP ==============================
    fig, ax = plt.subplots(1, 1, figsize = (15, 10), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines()
    ax.set_extent([-140, -50, 20, 55])
    cf = ax.contourf(
        hr['LONGXY'][0, :].values, hr['LATIXY'][:, 0].values,
        pct_crop, levels = np.linspace(0, 100., 21),
                    norm = BoundaryNorm(np.linspace(0, 100., 21), ncolors = 256, 
                                        clip = True), cmap = 'Reds')
    plt.colorbar(cf, ax = ax, shrink = 0.7)
    ax.set_title('PCT_CROP')
    """

    # Generate the vegetation map 2015, which is the last year
    year = np.where(hr['YEAR'] == year)[0][-1]
    pct_nat_pft = hr['PCT_NAT_PFT'][year, :, :, :].values
    area_natveg = hr['AREA'].values * pct_natveg / 100.

    # exclude oceans
    area_nat_pft = np.where(area_natveg > 0, np.full([6, 360, 720], 0), np.nan)
    area_nat_pft[0, :, :] = pct_nat_pft[0, :, :] / 100 * area_natveg
    area_nat_pft[1, :, :] = pct_nat_pft[[1,2], :, :].sum(axis = 0) / 100 * area_natveg
    area_nat_pft[2, :, :] = pct_nat_pft[[7,8], :, :].sum(axis = 0) / 100 * area_natveg
    area_nat_pft[3, :, :] = pct_nat_pft[[9,10,11], :, :].sum(axis = 0) / 100  * area_natveg
    area_nat_pft[4, :, :] = pct_nat_pft[[12,13,14], :, :].sum(axis = 0) / 100  * area_natveg
    area_nat_pft[5, :, :] = pct_nat_pft[15, :, :] / 100  * area_natveg

    # Limit to CONUS
    lat_filter = (hr['LATIXY'].values[:, 0] >= 20) & (hr['LATIXY'].values[:, 0] <= 55)
    lon_filter = (hr['LONGXY'].values[0, :] >= -140) & (hr['LONGXY'].values[0, :] <= -50)

    #print(np.nanmax(pct_nat_pft))
    #print(np.nanmax(pct_natveg))
    #print(np.nanmax(hr['AREA'].values[lat_filter, :][:, lon_filter]))

    # Reverse from S->N to N->S
    area_nat_pft = area_nat_pft[:, lat_filter, :][:, ::-1, lon_filter]

    # Save to file
    transform = from_origin(-140.0, 55.0, 0.5, 0.5)  # (lon_min, lat_max, pixel_width, pixel_height)
    crs = rasterio.crs.CRS.from_epsg(4326)

    # Create a new rasterio dataset
    new_dataset = rasterio.open(
        file_out,
        'w',
        driver='GTiff',
        height=area_nat_pft.shape[1],
        width=area_nat_pft.shape[2],
        count=area_nat_pft.shape[0],
        dtype=area_nat_pft.dtype,
        crs=crs,
        transform=transform,
    )

    new_dataset.write(area_nat_pft)

    new_dataset.close()

    hr.close()

In [3]:
netcdf_to_tif(
    os.path.join(path_surfdata, 'landuse.timeseries_0.5x0.5_hist_simyr1850-2015_c240308.nc'), 
    2015,
    os.path.join(os.environ['PROJDIR'], 'ERW_LDRD', 'results', 'LUC', 'vegetation_2015.tif')
)

In [4]:
for infile, outfile in zip([
    'landuse.timeseries_0.5x0.5_ssp2_rcp45_simyr2015-2100_c240408.nc', 
    'landuse.timeseries_0.5x0.5_ssp3_rcp70_simyr2015-2100_c240308.nc',
    'landuse.timeseries_0.5x0.5_ssp5_rcp85_simyr2015-2100_c240408.nc'
], ['vegetation_ssp2_rcp45', 'vegetation_ssp3_rcp70', 'vegetation_ssp5_rcp85']):
    for year in [2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055]:
        netcdf_to_tif(
            os.path.join(path_surfdata, infile), 
            year,
            os.path.join(os.environ['PROJDIR'], 'ERW_LDRD', 'results', 'LUC', 
                         f'{outfile}_{year}.tif')
        )